# Modelo: Regresion Logistica

En este notebook construiremos nuestro primer modelo de clasificación para la competencia de Kaggle UDEA/ai4eng 20252.
Utilizaremos un modelo de Regresión Logística, que servirá como baseline o línea base para comparar luego con modelos más complejos como Random Forest y XGBoost.

El objetivo principal de este cuaderno es:
* Entrenar un primer modelo simple.
* Evaluar su desempeño utilizando el conjunto de train.
* Obtener métricas iniciales (accuracy, matriz de confusión, clasificación).
* Generar un primer archivo de predicciones para Kaggle.


### Importaciones

En esta sección cargamos todas las librerías necesarias para entrenamiento, evaluación y generación de predicciones.

In [ ]:
import os, json
import zipfile
import pandas as pd
import numpy as np
from google.colab import userdata

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression

### Traemos los datos desde kaggle

In [5]:
user = userdata.get('KAGGLE_USERNAME')
key = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = user
os.environ["KAGGLE_KEY"] = key
assert user and key, "Faltan los secretos KAGGLE_USERNAME/KAGGLE_KEY"
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia
!unzip udea*.zip > /dev/null
!wc *.csv

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.38GB/s]
   296787    296787   4716673 submission_example.csv
   296787   4565553  59185238 test.csv
   692501  10666231 143732437 train.csv
  1286075  15528571 207634348 total


In [14]:
df_train = pd.read_csv("train.csv")
df_test  = pd.read_csv("test.csv")
df_sample = pd.read_csv("submission_example.csv")

df_train.head(3)

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264


### Preprocesado de la Data

Para el preprocesado usare la funcion propuesta para la entrega #2 con unas pequeñas adaptaciones

In [20]:
def make_scaler(kind: str):
    kind = (kind or 'standard').lower()
    if kind in ('standard', 'std', 'zscore'):
        return StandardScaler(with_mean=False)  # compatible con matrices dispersas
    else:
        return StandardScaler(with_mean=False)


def build_preprocessor_v2(
    df: pd.DataFrame,
    y_col: str = 'RENDIMIENTO_GLOBAL',
    id_col: str = 'ID',
    num_impute: str = 'median',
    cat_impute: str = 'most_frequent',
    encode_categorical: str = 'onehot',
    ordinal_maps: dict = None,
    scale_numeric: str = 'standard'
):
    data = df.copy()
    y = data.pop(y_col) if y_col in data.columns else None

    # Quitar columnas no predictoras
    for col in [id_col, 'Y_NUMERIC']:
        if col in data.columns:
            data = data.drop(columns=col)

    # Tipos
    num_cols = data.select_dtypes(include=['int64','float64']).columns.tolist()
    cat_cols = data.select_dtypes(include=['object']).columns.tolist()

    # Ordinal vs OneHot
    ordinal_maps = ordinal_maps or {}
    ordinal_cols = [c for c in cat_cols if c in ordinal_maps]
    onehot_cols  = [c for c in cat_cols if c not in ordinal_maps]

    # Pipelines
    num_pipe = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy=num_impute)),
        ('scaler', make_scaler(scale_numeric))
    ])

    onehot_pipe = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy=cat_impute)),
        ('onehot', OneHotEncoder(handle_unknown='infrequent_if_exist', sparse_output=True))
    ])

    ordinal_pipe = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy=cat_impute)),
        ('ordinal', OrdinalEncoder(
            categories=[ordinal_maps[c] for c in ordinal_cols],
            handle_unknown='use_encoded_value', unknown_value=-1
        ))
    ]) if ordinal_cols else 'drop'

    preprocessor = ColumnTransformer(
        transformers=[
            ('num',    num_pipe,    num_cols),
            ('onehot', onehot_pipe, onehot_cols),
            ('ordinal', ordinal_pipe, ordinal_cols)
        ],
        remainder='drop',
        n_jobs=1
    )
    return preprocessor, data, y

In [16]:
ordinal_maps = {
  'E_VALORMATRICULAUNIVERSIDAD': [
      'NO PAGO MATRICULA',
      'MENOS DE 500 MIL',
      'ENTRE 500 MIL Y MENOS DE 1 MILLON',
      'ENTRE 1 MILLON Y MENOS DE 2.5 MILLONES',
      'ENTRE 2.5 MILLONES Y MENOS DE 4 MILLONES',
      'ENTRE 4 MILLONES Y MENOS DE 5.5 MILLONES',
      'ENTRE 5.5 MILLONES Y MENOS DE 7 MILLONES',
      'MAS DE 7 MILLONES'
  ],
  'E_HORASSEMANATRABAJA': [
      '0',
      'MENOS DE 10 HORAS',
      'ENTRE 11 Y 20 HORAS',
      'ENTRE 21 Y 30 HORAS',
      'MAS DE 30 HORAS'
  ],
  'F_ESTRATOVIVIENDA': [
      'SIN ESTRATO', 'ESTRATO 1', 'ESTRATO 2', 'ESTRATO 3', 'ESTRATO 4', 'ESTRATO 5', 'ESTRATO 6'
  ],
  'F_EDUCACIONPADRE': [
      'NO SABE',
      'NINGUNO',
      'PRIMARIA INCOMPLETA',
      'PRIMARIA COMPLETA',
      'SECUNDARIA (BACHILLERATO) INCOMPLETA',
      'SECUNDARIA (BACHILLERATO) COMPLETA',
      'TECNICA O TECNOLOGICA INCOMPLETA',
      'TECNICA O TECNOLOGICA COMPLETA',
      'EDUCACION PROFESIONAL INCOMPLETA',
      'EDUCACION PROFESIONAL COMPLETA',
      'POSTGRADO'
    ],
    'F_EDUCACIONMADRE': [
      'NO SABE',
      'NINGUNO',
      'PRIMARIA INCOMPLETA',
      'PRIMARIA COMPLETA',
      'SECUNDARIA (BACHILLERATO) INCOMPLETA',
      'SECUNDARIA (BACHILLERATO) COMPLETA',
      'TECNICA O TECNOLOGICA INCOMPLETA',
      'TECNICA O TECNOLOGICA COMPLETA',
      'EDUCACION PROFESIONAL INCOMPLETA',
      'EDUCACION PROFESIONAL COMPLETA',
      'POSTGRADO'
  ]
}

In [21]:
preproc, X_df, y = build_preprocessor_v2(
    df_train,
    y_col='RENDIMIENTO_GLOBAL',
    id_col='ID',
    num_impute='median',
    cat_impute='most_frequent',
    encode_categorical='onehot',
    ordinal_maps=ordinal_maps,
    scale_numeric='standard'
)

In [22]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_df, y, test_size=0.2, stratify=y, random_state=42
)

### Entrenamiento del modelo de Regresión Logística

Entrenamos un LogisticRegression con hiperparámetros iniciales.
Este modelo es lineal y rápido, por lo que es útil para evaluar rápidamente si existen patrones lineales en el dataset.

In [26]:
logreg = Pipeline(steps=[
    ('prep', preproc),
    ('clf', LogisticRegression(
        multi_class='multinomial',
        solver='saga',
        max_iter=300,
        n_jobs=-1
    ))
])
logreg.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Pipeline(steps=[('prep',
                 ColumnTransformer(n_jobs=1,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler(with_mean=False))]),
                                                  ['PERIODO_ACADEMICO',
                                                   'INDICADOR_1', 'INDICADOR_2',
                                                   'INDICADOR_3',
                                                   'INDICADOR_4']),
                                                 ('onehot',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHo...
                                                                                               'EDUCACION '
                                                                                               'PROFESIONAL '
                                                                                               'INCOMPLETA',
                                                                                               'EDUCACION '
                                                                                               'PROFESIONAL '
                                                                                               'COMPLETA',
                                                                                               'POSTGRADO']],
                                                                                  handle_unknown='use_encoded_value',
                                                                                  unknown_value=-1))]),
                                                  ['E_VALORMATRICULAUNIVERSIDAD',
                                                   'E_HORASSEMANATRABAJA',
                                                   'F_ESTRATOVIVIENDA',
                                                   'F_EDUCACIONPADRE',
                                                   'F_EDUCACIONMADRE'])])),
                ('clf',
                 LogisticRegression(max_iter=300, multi_class='multinomial',
                                    n_jobs=-1, solver='saga'))])

### Evaluación del modelo

Incluimos:
* accuracy
* matriz de confusión
* classification report
* curva ROC (si aplica)

Estas métricas permiten visualizar el comportamiento del modelo y entender sus fortalezas y debilidades.

In [24]:
preds = logreg.predict(X_valid)
acc  = accuracy_score(y_valid, preds)
f1m  = f1_score(y_valid, preds, average='macro')

print(f'Accuracy: {acc:.4f} | F1-macro: {f1m:.4f}')
print('\nReporte:\n', classification_report(y_valid, preds))
print('\nMatriz de confusión:\n', confusion_matrix(y_valid, preds, labels=sorted(y.unique())))

Accuracy: 0.3704 | F1-macro: 0.3440

Reporte:
               precision    recall  f1-score   support

        alto       0.42      0.61      0.50     35124
        bajo       0.38      0.52      0.44     34597
  medio-alto       0.29      0.16      0.20     34324
  medio-bajo       0.30      0.19      0.23     34455

    accuracy                           0.37    138500
   macro avg       0.35      0.37      0.34    138500
weighted avg       0.35      0.37      0.34    138500


Matriz de confusión:
 [[21371  5397  4665  3691]
 [ 6694 17906  4130  5867]
 [13128  9805  5487  5904]
 [ 9327 13625  4960  6543]]
